## 1. 지수 스케일링

### 1) 치안 안전지수

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
peace = pd.read_excel("면적당 각종 지표 개수+인구.xlsx")

In [ ]:
light = peace["면적당 가로등 개수"]
bar = peace["면적당 유흥업소 개수"]
bell = peace["면적당 안전비상벨 개수"]
cctv = peace["면적당 CCTV 개수"]
police = peace["한 경찰이 담당하는 인구수"]

In [ ]:
# 스케일링 범위 설정
min_value = 1
max_value = 10

In [ ]:
# 가로등 안전지수 스케일링
data = light
light_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
print(light_scaled)

In [ ]:
# 유흥업소 안전지수 스케일링
data = bar
bar_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
bar_scaled = np.array(bar_scaled)

def reverse_score(score):
    reversed_score = 11 - score
    return reversed_score

bar_scaled_reverse = reverse_score(bar_scaled)
print(bar_scaled)
print(bar_scaled_reverse)


In [ ]:
# 안전비상벨 안전지수 스케일링
data = bell
bell_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
print(bell_scaled)

In [ ]:
# CCTV 안전지수 스케일링
data = cctv
cctv_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
print(cctv_scaled)

In [ ]:
# 경찰 행정력 안전지수 스케일링
data = police
police_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
police_scaled = np.array(police_scaled)

def reverse_score(score):
    reversed_score = 11 - score
    return reversed_score

police_scaled_reverse = reverse_score(police_scaled)
print(police_scaled)
print(police_scaled_reverse)


In [ ]:
peace["가로등 스케일 지수"] = light_scaled
peace["유흥업소 스케일 지수"] = bar_scaled_reverse
peace["안전비상벨 스케일 지수"] = bell_scaled
peace["CCTV 스케일 지수"] = cctv_scaled
peace["경찰 스케일 지수"] = police_scaled_reverse

In [ ]:
peace.to_excel("치안 데이터_최종완성본_827.xlsx")

### 2) 의료 지수

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hhh = pd.read_excel("병원인구최종.xlsx")

In [ ]:
hos = hhh["병원의 인구수용력"]
doc = hhh["한 의사당 담당 인구수"]

In [ ]:
# 병원지수에서 일봉동 삭제, 의사지수에서 일봉동,성남면 삭제
del hos[5]
del doc[5]
del doc[17]

In [ ]:
# 스케일링 범위
min_value = 1
max_value = 10

In [ ]:
# 병원지수 스케일링
data = hos
hos_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
hos_scaled = np.array(hos_scaled)

def reverse_score(score):
    reversed_score = 11 - score
    return reversed_score

hos_scaled_reverse = reverse_score(hos_scaled)
print(hos_scaled)
print(hos_scaled_reverse)

In [ ]:
# 삭제했던 값에 0 추가
hos_scaled_reverse = np.insert(hos_scaled_reverse, 5, 0)

In [ ]:
hos_scaled_reverse = pd.Series(hos_scaled_reverse)

In [ ]:
# 의사지수 스케일링
data = doc
doc_scaled = (data - np.min(data)) / (np.max(data) - np.min(data)) * (max_value - min_value) + min_value
doc_scaled = np.array(doc_scaled)

def reverse_score(score):
    reversed_score = 11 - score
    return reversed_score

doc_scaled_reverse = reverse_score(doc_scaled)
print(doc_scaled)
print(doc_scaled_reverse)

In [ ]:
# 삭제했던 값에 0 추가
doc_scaled_reverse = np.insert(doc_scaled_reverse, 5, 0)
doc_scaled_reverse = np.insert(doc_scaled_reverse, 17, 0)

In [ ]:
doc_scaled_reverse = pd.Series(doc_scaled_reverse)

In [ ]:
hhh["병원 지수"] = hos_scaled_reverse
hhh["의사 지수"] = doc_scaled_reverse

In [ ]:
hhh.to_excel("의료지수스케일링.xlsx")

## 2. 지오코딩

### 1) 주소 텍스트 - 위경도 좌표 반환

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
import json
import pandas as pd
import numpy as np
import time
import os

try :
    import xmltodict
except :
    !pip install xmltodict
    import xmltodict

In [ ]:
df = pd.read_excel("천안경찰서위치.xlsx")
df.head()

In [ ]:
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [ ]:
client_id = ''
client_pw = ''

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

In [ ]:
geo_coordi_list = []
road_add_list = []
jibun_add_list = []
address_ele_lit = []
i = 0
start_time = time.time()

for add in df['주소']:              ## 데이터명에 따라 변경
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)

    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode() # 정상이면 200 리턴
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                try:
                    road_address = response_body['addresses'][0]['roadAddress']
                    jibun_address = response_body['addresses'][0]['jibunAddress']
                    #address_elements = response_body['addresses'][0]['addressElements']
                    latitude = response_body['addresses'][0]['y']
                    longitude = response_body['addresses'][0]['x']
                    if i%100 == 0:
                        print(f'{i//100}00 results Success!')
                except:
                    road_address = None
                    jibun_address = None
                    #address_elements = None
                    latitude = None
                    longitude = None
            else:
                print('resulut not exist!')
                road_address = None
                jibun_address = None
                #address_elements = None
                latitude = None
                longitude = None
        else:
            print(f'Response error code: {rescode}')
            road_address = None
            jibun_address = None
            #address_elements = None
            latitude = None
            longitude = None

    geo_coordi_list.append([latitude, longitude])
    road_add_list.append(road_address)
    jibun_add_list.append(jibun_address)
    i += 1

print(f"총 소요시간: {time.time() - start_time: .2f}s")

In [ ]:
len(geo_coordi_list)

In [ ]:
geo_coordi_list

In [ ]:
np_geo_coordi = np.array(geo_coordi_list)
df_2 = df.copy()

df_2['road_address'] = road_add_list
df_2['jibun_address'] = jibun_add_list
df_2['latitude'] = np_geo_coordi[:, 0]
df_2['longitude'] = np_geo_coordi[:, 1]
df_2

In [ ]:
cheon_df_2.to_csv('천안시/천안시_읍면동사무소정보_0820.csv', encoding = 'utf-8-sig', index = False)

### 2) 행정동 매핑

좌표 위치 + shp polygon 조인

In [ ]:
df = pd.read_excel('병원정보서비스_천안_수정.xlsx')
df.columns

In [ ]:
cheon_shp = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/공모전/천안시/천안 shp/cheon_.shp', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
cheon_shp.head()

In [ ]:
cheon_shp.plot(color='gray', edgecolor="w")

In [ ]:
df.columns

In [ ]:
df['geometry'] = gpd.points_from_xy(df['좌표(X)'], df['좌표(Y)'], crs="EPSG:4326")
geo_df = gpd.GeoDataFrame(data=df, geometry='geometry')
geo_df = geo_df.to_crs(epsg=5181)

In [ ]:
geo_df_2 = gpd.sjoin(left_df=geo_df, right_df=cheon_shp, how='left', predicate='within')
geo_df_2

In [ ]:
geo_df_2.columns

In [ ]:
geo_df_2.drop(['geometry', 'index_right', 'BASE_DATE'], axis = 1, inplace = True)
fin_df = pd.DataFrame(geo_df_2)

In [ ]:
fin_df.columns

In [ ]:
fin_df.to_csv('천안/병원정보서비스_천안_수정_0827.csv', encoding = 'utf-8-sig', index = False)